In [2]:
# for data handling
import pandas as pd
import numpy as np

# train-test split
from sklearn.model_selection import train_test_split

# loss functions for today
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# stuff for evaluating classifiers
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt # for displaying a pretty confusion matrix

# dummy models for comparison
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier

# regression models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# classification models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# visual models
import seaborn as sns
import matplotlib.pyplot as plt

Due to an error in the dataset the airport identifiers from the entirety of October are compromised of intergers instead of the airport ID. This is quite troublesome to fix which is why we've decided to remove the rows from October even though this will defininetly decrease the accuracy of our model.

In [126]:
flight_df = pd.read_csv('flights.csv', dtype={'DESTINATION_AIRPORT': str, 'ORIGIN_AIRPORT': str})

In [127]:
#flight_df_new = flight_df.copy()
#remove_october = flight_df_new[flight_df_new['MONTH'] == 10]

#flight_df_filtered = flight_df_new[~remove_october]
#flight_df_filtered = flight_df.drop(flight_df[remove_october].index)

flight_df = flight_df[flight_df['MONTH'] != 10]

#flight_df = flight_df[flight_df['MONTH'] == 10]

flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
flight_df = flight_df.sample(n=50000, random_state=42)

flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4184260,2015,9,18,5,B6,23,N923JB,JFK,LAX,700,...,951.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1523550,2015,4,8,3,US,1745,N838AW,IAH,CLT,1205,...,1540.0,10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3257379,2015,7,23,4,MQ,3384,N642MQ,MLI,ORD,555,...,648.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4128422,2015,9,14,1,WN,1913,N478WN,LAS,IAD,1530,...,2253.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3025631,2015,7,9,4,EV,5165,N582CA,ATL,GRK,1030,...,1125.0,-19.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


# Exploratory

In [15]:
flight_df.shape

(50000, 31)

In [16]:
flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   YEAR                 50000 non-null  int64  
 1   MONTH                50000 non-null  int64  
 2   DAY                  50000 non-null  int64  
 3   DAY_OF_WEEK          50000 non-null  int64  
 4   AIRLINE              50000 non-null  object 
 5   FLIGHT_NUMBER        50000 non-null  int64  
 6   TAIL_NUMBER          49941 non-null  object 
 7   ORIGIN_AIRPORT       50000 non-null  object 
 8   DESTINATION_AIRPORT  50000 non-null  object 
 9   SCHEDULED_DEPARTURE  50000 non-null  int64  
 10  DEPARTURE_TIME       48896 non-null  float64
 11  DEPARTURE_DELAY      48896 non-null  float64
 12  TAXI_OUT             48873 non-null  float64
 13  WHEELS_OFF           48873 non-null  float64
 14  SCHEDULED_TIME       50000 non-null  int64  
 15  ELAPSED_TIME         48753 non-null 

In [18]:
flight_df['ARRIVAL_DELAY'].max()

1384.0

Potential outlier - remove?

# Preprocessing

According to the FAA a flight is considered delayed it is delayed by 15 minutes or more.

In [136]:
df_fd = flight_df.copy()
df_fd = df_fd[df_fd.ARRIVAL_DELAY > 15]

We'll clear out some irrelevant columns and retain those we need for our prediction analysis.

In [137]:
# We will not consider cancelled or diverted flights as we are only concerned with delays
#df_fd = df_fd[df_fd.CANCELLED != 1]
#df_fd = df_fd[df_fd.DIVERTED != 1]

# Drop all columns that do not add interpretability to the model
df_fd = df_fd.drop(['YEAR', 'DEPARTURE_DELAY', 'DEPARTURE_TIME', 'TAXI_OUT','WHEELS_OFF', 'ELAPSED_TIME', 'AIR_TIME',
                  'WHEELS_ON','TAXI_IN','ARRIVAL_TIME','DIVERTED','CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY',
                   'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'], axis=1)

# Drop TAIL_NUMBER for now due to value error
df_fd = df_fd.drop(['TAIL_NUMBER', 'FLIGHT_NUMBER', 'DISTANCE'], axis=1)
    
# Fill NaNs - just to make sure
#df_fd = df_fd.fillna(0)

df_fd.head(30)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,SCHEDULED_ARRIVAL,ARRIVAL_DELAY
5074433,11,13,5,UA,SAN,SFO,1606,99.0,1745,92.0
3592254,8,11,2,DL,DEN,SLC,1915,92.0,2047,53.0
2902520,7,1,3,EV,ICT,ORD,1756,117.0,1953,145.0
3405158,7,31,5,EV,PBI,IAH,1705,164.0,1849,33.0
4318851,9,26,6,DL,MSP,ANC,1735,354.0,2029,56.0
2651251,6,16,2,MQ,XNA,ORD,1914,111.0,2105,35.0
1820738,4,26,7,WN,HOU,IND,2035,135.0,2350,24.0
5301306,11,28,6,WN,BWI,MDW,1755,130.0,1905,26.0
5243421,11,24,2,AA,SFO,DFW,1540,204.0,2104,78.0
3440759,8,2,7,WN,ATL,MSP,2100,160.0,2240,21.0


Encoding

In [138]:
columns_to_encode = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']

df_fd = pd.get_dummies(df_fd, columns=columns_to_encode,  drop_first=True)

df_fd.head()

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,...,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YUM
5074433,11,13,5,1606,99.0,1745,92.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3592254,8,11,2,1915,92.0,2047,53.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2902520,7,1,3,1756,117.0,1953,145.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3405158,7,31,5,1705,164.0,1849,33.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4318851,9,26,6,1735,354.0,2029,56.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [62]:
df_fd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14871 entries, 27 to 49997
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MONTH                14871 non-null  int64  
 1   DAY                  14871 non-null  int64  
 2   DAY_OF_WEEK          14871 non-null  int64  
 3   AIRLINE              14871 non-null  int32  
 4   ORIGIN_AIRPORT       14871 non-null  int32  
 5   DESTINATION_AIRPORT  14871 non-null  int32  
 6   SCHEDULED_DEPARTURE  14871 non-null  int64  
 7   SCHEDULED_TIME       14871 non-null  int64  
 8   SCHEDULED_ARRIVAL    14871 non-null  int64  
 9   ARRIVAL_DELAY        14871 non-null  float64
dtypes: float64(1), int32(3), int64(6)
memory usage: 1.1 MB


In [73]:
df_fd.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,SCHEDULED_ARRIVAL,ARRIVAL_DELAY
27,1,1,4,8,193,95,115,207,542,25.0
30,1,1,4,8,211,199,125,204,549,43.0
50,1,1,4,2,34,167,307,173,500,20.0
52,1,1,4,2,252,167,400,185,605,85.0
55,1,1,4,2,252,21,438,241,739,89.0


Train-Test split

In [139]:
y = df_fd['ARRIVAL_DELAY']
X = df_fd.drop('ARRIVAL_DELAY', axis=1)

#X = df_fd[['MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'SCHEDULED_ARRIVAL']]

#df_fd_enc[['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
#          'SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL']]

Box plot visual

In [32]:
df_melted = pd.melt(df_fd, id_vars=X, value_vars=[y])

# Create a box plot using Seaborn
plt.figure(figsize=(10, 6))
sns.boxplot(x='variable', y='value', data=df_melted)
plt.title('Box Plot of Y by Categories in X')
plt.xlabel('X Categories')
plt.ylabel('Y Values')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.show()

KeyError: "The following 'value_vars' are not present in the DataFrame: [16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 160

Creating training and test size

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

# Regression

Code should be working, however requires a lot of memory unless we downsize the amount of rows.

In [141]:
fd_ln = LinearRegression()

fd_ln.fit(X_train, y_train)

LinearRegression()

In [142]:
print("Score: {:.3f}".format(fd_ln.score(X, y)))

Score: 0.057
